In [1]:
#From # https://docs.quantum.ibm.com/guides/build-noise-models
#IBM Quantum support - AC 2024 10 02

#!/usr/bin/env python3
__author__ = "Jan Balewski"
__email__ = "janstar1122@gmail.com"

'''
example of density matrix computation using Qiskit 1.2
'''
from qiskit import  QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.quantum_info.operators import Operator
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from pprint import pprint

In [2]:
# iSWAP matrix operator
iswap_op = Operator([[1, 0, 0, 0],
                     [0, 0, 1j, 0],
                     [0, 1j, 0, 0],
                     [0, 0, 0, 1]])
# Using operators in circuits
nq=2;nb=2
qc = QuantumCircuit(2)

# Add gates
qc.sdg(1)
qc.h(1)
qc.sdg(0)
qc.unitary(iswap_op, [0, 1], label='iswap')
qc.sdg(0)
qc.save_density_matrix()
qc.sdg(0)
qc.unitary(iswap_op, [0, 1], label='iswap')
qc.s(1)

print(qc)

     ┌─────┐     ┌────────┐┌─────┐ density_matrix ┌─────┐┌────────┐     
q_0: ┤ Sdg ├─────┤0       ├┤ Sdg ├───────░────────┤ Sdg ├┤0       ├─────
     ├─────┤┌───┐│  iswap │└─────┘       ░        └─────┘│  iswap │┌───┐
q_1: ┤ Sdg ├┤ H ├┤1       ├──────────────░───────────────┤1       ├┤ S ├
     └─────┘└───┘└────────┘              ░               └────────┘└───┘


In [3]:
backend1 = AerSimulator(method="density_matrix")
print('job started,  nq=%d  at %s ...'%(qc.num_qubits,backend1.name))

job = backend1.run(qc)
result = job.result() 
#!pprint(result)
print('ideal density matrix:',result.data())

job started,  nq=2  at aer_simulator_density_matrix ...
ideal density matrix: {'density_matrix': DensityMatrix([[0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j],
               [0.5+0.j, 0.5+0.j, 0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]],
              dims=(2, 2))}


In [4]:
print('\n repeat on  fake backend ...')
service = QiskitRuntimeService(channel="ibm_quantum")

backName='ibm_nazca'   # EPLG=3.2%

## Building noise model from existing hardware is described here :
# https://docs.quantum.ibm.com/guides/build-noise-models
# https://docs.quantum.ibm.com/guides/build-noise-models#initializing-a-noise-model-from-a-backend

# Import from Qiskit Aer noise module
from qiskit_aer.noise import NoiseModel
noisy_backend = service.backend(backName)

# Create noisy simulator backend
noise_model = NoiseModel.from_backend(noisy_backend)

backend2 = AerSimulator(method="density_matrix", noise_model=noise_model)


 repeat on  fake backend ...


In [5]:
#print(noise_model)

In [6]:
# Transpiling
circ = transpile(qc, backend2)

In [7]:
job2 = backend2.run(circ)

In [8]:
#job2 = backend2.run(qcT)
result2 = job2.result() 
#print('noisy density matrix:',result2.data())

In [9]:
noisy_density_matrix = result2.data()
matrix = noisy_density_matrix['density_matrix']

In [10]:
print(noisy_density_matrix['density_matrix'])

DensityMatrix([[ 4.89441139e-01+8.03531555e-21j,
                 4.76946455e-01+1.07414257e-03j,
                 1.81525056e-03+1.38870306e-03j,
                 4.18982330e-05+1.32717589e-03j],
               [ 4.76946455e-01-1.07414257e-03j,
                 4.89254719e-01-8.03531555e-21j,
                -4.18982330e-05+1.32213101e-03j,
                 9.11492867e-04+1.38816043e-03j],
               [ 1.81525056e-03-1.38870306e-03j,
                -4.18982330e-05-1.32213101e-03j,
                 1.06565681e-02+1.38640334e-17j,
                 1.71004383e-04-1.47993693e-04j],
               [ 4.18982330e-05-1.32717589e-03j,
                 9.11492867e-04-1.38816043e-03j,
                 1.71004383e-04+1.47993693e-04j,
                 1.06475735e-02-1.38640334e-17j]],
              dims=(2, 2))


In [11]:
import numpy as np
from sympy import Matrix
Matrix(matrix)

Matrix([
[ 0.489441138975257 + 8.03531555312319e-21*I,    0.476946455299398 + 0.00107414256941457*I,  0.00181525055844887 + 0.00138870306484384*I,  4.18982330476351e-5 + 0.00132717589030297*I],
[  0.476946455299399 - 0.00107414256941457*I,   0.489254719399056 - 8.03531555312242e-21*I, -4.18982330474688e-5 + 0.00132213100808392*I,  0.00091149286662448 + 0.00138816042651309*I],
[0.00181525055844885 - 0.00138870306484384*I, -4.18982330474688e-5 - 0.00132213100808393*I,   0.010656568113963 + 1.38640334469468e-17*I, 0.000171004383484828 - 0.00014799369284617*I],
[  4.18982330476351e-5 - 0.001327175890303*I, 0.000911492866624493 - 0.00138816042651309*I, 0.000171004383484814 + 0.00014799369284617*I,  0.0106475735117235 - 1.38640334469468e-17*I]])

In [13]:
print(circ.draw('text', idle_wires=False))

global phase: 3π/2
     ┌──────────┐┌────┐┌──────┐┌─────────┐ ┌────┐┌──────────┐┌──────┐┌────┐»
q_0: ┤ Rz(-π/2) ├┤ √X ├┤0     ├┤ Rz(π/2) ├─┤ √X ├┤ Rz(-π/2) ├┤0     ├┤ √X ├»
     └──┬───┬───┘└────┘│  Ecr │├─────────┴┐├────┤├─────────┬┘│  Ecr │├────┤»
q_1: ───┤ X ├──────────┤1     ├┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├─┤1     ├┤ √X ├»
        └───┘          └──────┘└──────────┘└────┘└─────────┘ └──────┘└────┘»
«      density_matrix ┌──────────┐┌────┐┌──────┐┌─────────┐ ┌────┐┌──────────┐»
«q_0: ───────░────────┤ Rz(-π/2) ├┤ √X ├┤0     ├┤ Rz(π/2) ├─┤ √X ├┤ Rz(-π/2) ├»
«            ░        ├──────────┤├────┤│  Ecr │├─────────┴┐├────┤├─────────┬┘»
«q_1: ───────░────────┤ Rz(-π/2) ├┤ √X ├┤1     ├┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├─»
«            ░        └──────────┘└────┘└──────┘└──────────┘└────┘└─────────┘ »
«     ┌──────┐┌────┐┌─────────┐
«q_0: ┤0     ├┤ √X ├┤ Rz(π/2) ├
«     │  Ecr │├────┤├─────────┤
«q_1: ┤1     ├┤ √X ├┤ Rz(π/2) ├
«     └──────┘└────┘└─────────┘


In [14]:
pm = generate_preset_pass_manager(optimization_level=1, backend=backend2)
qcT = pm.run(qc)
print('transpiled for',backend2)
print(qcT.draw('text', idle_wires=False))


job2 = backend2.run(qcT)
result2 = job2.result() 
print('noisy density matrix:',result2.data())


transpiled for AerSimulator('aer_simulator_density_matrix'
             noise_model=<NoiseModel on ['ecr', 'sx', 'measure', 'x', 'reset', 'id']>)
global phase: 3π/2
     ┌──────────┐┌────┐┌──────┐┌─────────┐ ┌────┐┌──────────┐┌──────┐┌────┐»
q_0: ┤ Rz(-π/2) ├┤ √X ├┤0     ├┤ Rz(π/2) ├─┤ √X ├┤ Rz(-π/2) ├┤0     ├┤ √X ├»
     └──┬───┬───┘└────┘│  Ecr │├─────────┴┐├────┤├─────────┬┘│  Ecr │├────┤»
q_1: ───┤ X ├──────────┤1     ├┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├─┤1     ├┤ √X ├»
        └───┘          └──────┘└──────────┘└────┘└─────────┘ └──────┘└────┘»
«      density_matrix ┌──────────┐┌────┐┌──────┐┌─────────┐ ┌────┐┌──────────┐»
«q_0: ───────░────────┤ Rz(-π/2) ├┤ √X ├┤0     ├┤ Rz(π/2) ├─┤ √X ├┤ Rz(-π/2) ├»
«            ░        ├──────────┤├────┤│  Ecr │├─────────┴┐├────┤├─────────┬┘»
«q_1: ───────░────────┤ Rz(-π/2) ├┤ √X ├┤1     ├┤ Rz(-π/2) ├┤ √X ├┤ Rz(π/2) ├─»
«            ░        └──────────┘└────┘└──────┘└──────────┘└────┘└─────────┘ »
«     ┌──────┐┌────┐┌─────────┐
«q_0: ┤0     ├┤ √X